# Reconnaissance de caractères et de symboles

In [38]:
import numpy as np
import cv2
import os

In [39]:
# détécteur de codes barres
bd = cv2.barcode_BarcodeDetector()

# détécteur de QR codes
qd = cv2.QRCodeDetector()

In [40]:
def detect_barcode(img_path):
    img = cv2.imread(img_path)
    return bd.detectAndDecode(img)

In [41]:
def detect_qrcode(img_path):
    img = cv2.imread(img_path)
    return qd.detectAndDecodeMulti(img)

In [42]:
def test_barcode_detection():
    EMPTY_IMAGES_PATH = './data/images/empty'
    BARCODE_IMAGES_PATH = './data/images/barcode'

    # boucle sur les dossiers d'images
    for path in [EMPTY_IMAGES_PATH, BARCODE_IMAGES_PATH]:
        # boucle sur les images du dossier
        for img_path in os.listdir(path):
            print(path +'/' + img_path)
            print(detect_barcode(path +'/' + img_path))

        print('\n')

In [43]:
def test_qrcode_detection():
    EMPTY_IMAGES_PATH = './data/images/empty'
    BARCODE_IMAGES_PATH = './data/images/qrcode'

    # boucle sur les dossiers d'images
    for path in [EMPTY_IMAGES_PATH, BARCODE_IMAGES_PATH]:
        # boucle sur les images du dossier
        for img_path in os.listdir(path):
            print(path +'/' + img_path)
            print(detect_qrcode(path +'/' + img_path))

        print('\n')

In [44]:
test_barcode_detection()

./data/images/empty/img_001.jpg
('', None, None)


./data/images/barcode/img_001.png
('073935942400', None, None)




In [45]:
test_qrcode_detection()

./data/images/empty/img_001.jpg
(False, (), None, ())


./data/images/qrcode/SEM_Blog_body_image-445x510-IMG_0976-QRscan.png
(False, (), None, ())


